In [1]:
import pandas as pd
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report



In [2]:
userData = pd.read_csv('../data/userData.csv')
userData = userData.drop(columns=['weekday'])
userData

,food_name,time_of_day
0,sweet tea,Night
1,chick-fil-a sandwich,Night
2,chick-fil-a sandwich,Night
3,chick-fil-a 8 count nugget,Night
4,chocolate chip muffin,Night
...,...,...
359,optimum nutrition scoop of whey,Evening
360,cups of milk,Evening
361,bags of oatmeal,Night
362,optimum nutrition shake with 2 cups of milk,Evening


In [3]:

# Assuming you have your data loaded into a DataFrame called 'userData'
# Let's encode categorical variables using one-hot encoding
df_encoded = pd.get_dummies(userData, columns=['time_of_day'])

# Separate features and target variable
X = df_encoded.drop('food_name', axis=1)
y = df_encoded['food_name']


times_of_day = ['Morning', 'Afternoon', 'Evening', 'Night']

# Generate all possible combinations of day and time
all_permutations = list(times_of_day)

# print(all_permutations)

# Create a DataFrame to store the permutations
manual_test = pd.DataFrame(all_permutations, columns=['time_of_day'])
manual_encoded = pd.get_dummies(manual_test, columns=['time_of_day'])



# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=43)

# Initialize the Multi-Layer Perceptron (MLP) classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=43)

# Train the classifier
mlp_classifier.fit(X_train, y_train)

# Make predictions on the test set with probability estimates
y_pred_proba = mlp_classifier.predict_proba(manual_encoded)

# Specify how many top predictions you want to consider
top_n = 5
# Get the indices of the top N predictions for each sample
top_n_indices = (-y_pred_proba).argsort(axis=1)[:, :top_n]

# Get the top N predictions for each sample
top_n_predictions = mlp_classifier.classes_[top_n_indices]

# Assuming you have stored the day and time_of_day columns separately
# day_cols = ['weekday_Monday', 'weekday_Tuesday', 'weekday_Wednesday', 'weekday_Thursday', 'weekday_Friday', 'weekday_Saturday', 'weekday_Sunday']
time_cols = ['time_of_day_Morning', 'time_of_day_Afternoon', 'time_of_day_Evening', 'time_of_day_Night']

# Print the top N predictions for each sample along with their parameters and probabilities

result = []

with open("../out/time_based_output_mlp.txt", "w") as f:
    for i in range(len(manual_encoded)):
        # day = [col.split('_')[1] for col in day_cols if manual_encoded.iloc[i][col] == 1][0]
        time_of_day = [col.split('_')[3] for col in time_cols if manual_encoded.iloc[i][col] == 1][0]

        # f.write("Day: {}\n".format(day))
        f.write("Time of Day: {}\n".format(time_of_day))
        f.write("Predictions:\n")
        for j in range(top_n):
            f.write("    - Prediction {}: {}, Probability: {:.2f}%\n".format(j + 1, top_n_predictions[i][j], y_pred_proba[i][top_n_indices[i][j]] * 100))
        f.write("\n")
        result.append({
            # 'day': day,
            'time_of_day': time_of_day,
        })
        for j in range(top_n):  # Assuming you want top 5 predictions
            result[-1][f'prediction{j+1}'] = top_n_predictions[i][j]
            result[-1][f'probability{j+1}'] = y_pred_proba[i][top_n_indices[i][j]] * 100

df_result = pd.DataFrame(result)
df_result

# df_result.to_csv('time_based_results.csv', index=False)




,time_of_day,prediction1,probability1,prediction2,probability2,prediction3,probability3,prediction4,probability4,prediction5,probability5
0,Morning,fairlife chocolate milk,9.458011,4oz chicken,9.241703,venti caramel macchiato,9.222971,sweet tea,8.958496,kodiak cake waffles,8.915151
1,Afternoon,protein shake with milk and peanut butter,5.227854,cups of milk,5.001467,fairlife 2 percent chocolate milk,4.895685,bucees monster breakfast burrito,4.861163,optimum nutrition protein shake,4.841776
2,Evening,chick-fil-a sandwich,12.869776,sweet tea,10.946076,kodiak cake waffles,3.500009,borden chocolate milk,2.403362,central market blackened chicken caesar wrap,2.379874
3,Night,chick-fil-a sandwich,27.358301,sweet tea,23.005266,milk,1.981333,chick-fil-a granola parfait,1.885713,banana,1.863594
